# Elasticsearchを利用したサンプル検索

Elasticsearchを利用した検索などを試すためのノートブックです。

## 接続先の設定

**ESHOST**に接続先を設定します。サンプルでは、ローカルネットワーク環境にあるElasticsearchに接続しています。
接続先の各種設定については公式ドキュメントをご覧ください。
https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html

**INDEX**が接続先のインデックス名です。

In [4]:
from elasticsearch import Elasticsearch
import json

ESHOST = "http://192.168.1.240:9200"
INDEX = "esci-raw-data"

esclient = Elasticsearch(ESHOST)

## スキーマの確認

対象となるインデックスのスキーマを確認します。

In [11]:
import json
response = esclient.indices.get(index=INDEX, pretty=True)
print(json.dumps(response.body, indent=2))

{
  "esci-raw-data": {
    "aliases": {},
    "mappings": {
      "properties": {
        "product_brand": {
          "type": "keyword",
          "ignore_above": 256
        },
        "product_bullet_point": {
          "type": "keyword",
          "ignore_above": 256,
          "fields": {
            "en": {
              "type": "text",
              "analyzer": "standard"
            },
            "ja": {
              "type": "text",
              "analyzer": "kuromoji"
            }
          }
        },
        "product_color": {
          "type": "keyword"
        },
        "product_description": {
          "type": "keyword",
          "ignore_above": 256,
          "fields": {
            "en": {
              "type": "text",
              "analyzer": "standard"
            },
            "ja": {
              "type": "text",
              "analyzer": "kuromoji"
            }
          }
        },
        "product_id": {
          "type": "keyword"
        },
        "

## データの確認

検索してみる。

In [14]:
import pandas as pd

response = esclient.search(
    index=INDEX
)

result_df = pd.DataFrame(d["_source"] for d in response["hits"]["hits"])
result_df

,product_brand,product_bullet_point,product_color,product_description,product_id,product_locale,product_title
0,11 Degrees,11 Degrees Negro Playera con logo\nA estrenar ...,Negro,Esta playera con el logo de la marca Carrier d...,B079VKKJN7,es,"11 Degrees de los Hombres Playera con Logo, Ne..."
1,11 Degrees,None,Blanco,None,B079Y9VRKS,es,Camiseta Eleven Degrees Core TS White (M)
2,11 Degrees,11 Degrees Azul Core Pull Over Hoodie\nA estre...,Azul,La sudadera con capucha Core Pull Over de 11 G...,B07DP4LM9H,es,11 Degrees de los Hombres Core Pull Over Hoodi...
3,11 Degrees,None,None,None,B07G37B9HP,es,11 Degrees Poli Panel Track Pant XL Black
4,11 Degrees,None,Negro (,None,B07LCTGDHY,es,11 Degrees Gorra Trucker Negro OSFA (Talla úni...
5,11 Degrees,11 Degrees Negro Optum Poly Joggers\nA estrena...,Negro,Los Optum Poly Joggers de 11 grados vienen con...,B07MSD1JH3,es,"11 Degrees de los Hombres Optum Poly Joggers, ..."
6,11 Degrees,None,Negro,El Chándal ha sido diseñado con mangas largas ...,B07QKLGMHM,es,11 Degrees Core Zip Poly Top S Black
7,11 Degrees,None,None,None,B07S1VM815,es,11 Degrees Camiseta De Núcleo M Hot Red
8,11 Degrees,None,Black & White,None,B07T1HCDXG,es,11 Degrees Trucker Cap - Black & White
9,11 Degrees,11 Degrees Negro Chaqueta acolchada\nA estrena...,Negro,La chaqueta Space Puffer de 11 Degrees viene c...,B07VCV1LSQ,es,11 Degrees Chaqueta Espacial S Black


## クエリを指定して検索


In [ ]:
query = {
    "match": {
        "product_locale": "jp"
    }
}

response = esclient.search(
    index=INDEX,
    query=query
)

result_df = pd.DataFrame(d["_source"] for d in response["hits"]["hits"])
result_df